In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
from google.colab import files
uploaded = files.upload()

Saving SpotifyFeatures.csv to SpotifyFeatures.csv


In [0]:

#Data Characteristics
spotify_data = pd.read_csv("SpotifyFeatures.csv")
display(spotify_data.head())
display(spotify_data.info())
display(spotify_data.describe())
print("\nUnique genres: ", pd.unique(spotify_data['genre']))


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             232725 non-null  object 
 1   artist_name       232725 non-null  object 
 2   track_name        232725 non-null  object 
 3   track_id          232725 non-null  object 
 4   popularity        232725 non-null  int64  
 5   acousticness      232725 non-null  float64
 6   danceability      232725 non-null  float64
 7   duration_ms       232725 non-null  int64  
 8   energy            232725 non-null  float64
 9   instrumentalness  232725 non-null  float64
 10  key               232725 non-null  object 
 11  liveness          232725 non-null  float64
 12  loudness          232725 non-null  float64
 13  mode              232725 non-null  object 
 14  speechiness       232725 non-null  float64
 15  tempo             232725 non-null  float64
 16  time_signature    23

None

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,232725.000000,232725.000000,232725.000000,2.327250e+05,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000
mean,41.127502,0.368560,0.554364,2.351223e+05,0.570958,0.148301,0.215009,-9.569885,0.120765,117.666585,0.454917
std,18.189948,0.354768,0.185608,1.189359e+05,0.263456,0.302768,0.198273,5.998204,0.185518,30.898907,0.260065
min,0.000000,0.000000,0.056900,1.538700e+04,0.000020,0.000000,0.009670,-52.457000,0.022200,30.379000,0.000000
25%,29.000000,0.037600,0.435000,1.828570e+05,0.385000,0.000000,0.097400,-11.771000,0.036700,92.959000,0.237000
50%,43.000000,0.232000,0.571000,2.204270e+05,0.605000,0.000044,0.128000,-7.762000,0.050100,115.778000,0.444000
75%,55.000000,0.722000,0.692000,2.657680e+05,0.787000,0.035800,0.264000,-5.501000,0.105000,139.054000,0.660000
max,100.000000,0.996000,0.989000,5.552917e+06,0.999000,0.999000,1.000000,3.744000,0.967000,242.903000,1.000000



Unique genres:  ['Movie' 'R&B' 'A Capella' 'Alternative' 'Country' 'Dance' 'Electronic'
 'Anime' 'Folk' 'Blues' 'Opera' 'Hip-Hop' "Children's Music"
 'Children’s Music' 'Rap' 'Indie' 'Classical' 'Pop' 'Reggae' 'Reggaeton'
 'Jazz' 'Rock' 'Ska' 'Comedy' 'Soul' 'Soundtrack' 'World']


In [0]:
def one_hot_encode(df, column_name):
    """
    Function to one hot encode a column and remove the old column from data frame
    :param df: pandas data frame that needs encoding
    :param column_name: name of column
    :returns: pandas data frame with one hot encoded columns
    """
    df_one_hot = df.copy()
    df_one_hot = pd.concat([df_one_hot,pd.get_dummies(df_one_hot[column_name], prefix=column_name)],axis=1)
    df_one_hot.drop([column_name],axis=1, inplace=True)
    return df_one_hot



In [0]:
#Factorized Data
#Factorize columns that are strings
def factorized_column(df, column_name):
    """
    Function to factorize a column in a pandas dataframe
    :param df: pandas data frame that needs encoding
    :param column_name: name of column
    :returns: pandas data frame with factorized column and mapping of encoding
    """
    df_factorized = df.copy()
    df_factorized[column_name], mapping = pd.factorize(df[column_name])
    return df_factorized, mapping



In [0]:
#Checking for incomplete data
print("Amount of Null in Data: ", spotify_data.isnull().sum().sum())
print("Amount of N/A in Data: ", spotify_data.isna().sum().sum())
print("No incomplete data")

#Encoding Data as per Encoding Section
spotify_data_encoded = spotify_data.copy()
spotify_data_encoded = one_hot_encode(spotify_data_encoded,'key')
spotify_data_encoded = one_hot_encode(spotify_data_encoded,'genre')

mode_dictionary = {'Major' : 1, 'Minor' : 0}
spotify_data_encoded['mode'].replace(mode_dictionary, inplace=True)
time_dictionary = {'0/4' : 0, '1/4' : 1, '3/4': 1, '4/4': 4,'5/4': 5}
spotify_data_encoded['time_signature'].replace(time_dictionary, inplace=True)
print("Encoded key, genre, mode, time_signature")

genre_column_list = []
for genre in pd.unique(spotify_data['genre']):
    genre_column_list.append("genre_"+genre)

#Deleting Duplicates in data with exact values in track_id
#Spotify will categorize 1 unique song into multiple genres
#We will take the first genre that the song is categorized into
spotify_data_encoded = spotify_data_encoded.drop_duplicates(subset='track_id', keep='first')

#Consolodating duplicate tracks to 1 row with one hot encoding of genres
#Will take some time
print("Starting to make track dictionary")
trackid_to_genre_dict = {}
for index, row in spotify_data.iterrows():
    track_id = row['track_id']
    genre = row['genre']
    if track_id in trackid_to_genre_dict:
        trackid_to_genre_dict[track_id].append(genre)
    else:
        trackid_to_genre_dict[track_id] = [genre]
#display(trackid_to_genre_dict)
print("Done making track dictionary")

print("Starting to condense tracks to encoded dataframe")

for index, row in spotify_data_encoded.iterrows():    
    track_id = row['track_id']
    genre_list = trackid_to_genre_dict[track_id]
    for genre in genre_list:
        genre_label = "genre_"+ genre
        spotify_data_encoded.at[index,genre_label] = 1

print("Done condensing tracks to encoded dataframe")

print("Total Songs that were removed due to duplication: ", (spotify_data.shape[0] - spotify_data_encoded.shape[0]))
spotify_data_encoded.drop(['track_name'],axis=1, inplace=True)
spotify_data_encoded.drop(['track_id'],axis=1, inplace=True)
spotify_data_encoded.drop(['artist_name'],axis=1, inplace=True)



"""
#Failed attempts
#Takes way too long

unique_track_list = spotify_data_encoded['track_id']
for track in unique_track_list:
    query_df = spotify_data_encoded.query('track_id == "'+ str(track) + '"')
    for genre in genre_column_list:
        has_true = (query_df[genre] == 1).tolist()
        if True in has_true:
            for index in query_df.index.values.tolist():
                spotify_data_encoded.at[index,genre]=1

Another attempt to get rows done
for index, row in spotify_data.iterrows():
    if(index%1000 == 0):
        print(index)
    track_id = row['track_id']
    index_track = spotify_data_non_duplicates.index[spotify_data_non_duplicates['track_id'] == track_id].tolist()[0]
    genre = row['genre']
    genre_label = "genre_"+ genre
    spotify_data_non_duplicates.at[index_track,genre_label] = 1
"""
print("\nspotify_data_encoded is clean data")
print("\nspotify_data is original data")


Amount of Null in Data:  0
Amount of N/A in Data:  0
No incomplete data
Encoded key, genre, mode, time_signature
Starting to make track dictionary
Done making track dictionary
Starting to condense tracks to encoded dataframe
Done condensing tracks to encoded dataframe
Total Songs that were removed due to duplication:  55951

spotify_data_encoded is clean data

spotify_data is original data


In [0]:
#Standardizing the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

spotify_features = spotify_data_encoded.drop(genre_column_list,axis=1)
spotify_target = spotify_data_encoded[genre_column_list]

scaler.fit(spotify_features)
spotify_feature_scaler = scaler.transform(spotify_features)
spotify_feature_scaler = pd.DataFrame(spotify_feature_scaler, index=spotify_features.index, columns=spotify_features.columns)


Starting the Multi Level KNN From here

In [0]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 3.7MB/s 


In [0]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import train_test_split
classifier = MLkNN(k=3) 
X_train, X_test, y_train, y_test = train_test_split(spotify_feature_scaler, spotify_target, random_state = 42, test_size=0.33)
X_train_array = X_train.to_numpy()
X_test_array = X_test.to_numpy()
y_train_array = y_train.to_numpy()
y_test_array = y_test.to_numpy()

In [0]:
!pip install mlxtend

In [0]:
from sklearn.model_selection import train_test_split
#display(spotify_feature_scaler)
x = spotify_feature_scaler[['popularity', 'speechiness']]
y = spotify_features
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 42, test_size=0.33)
#X_train_array = X_train.to_numpy()
#X_test_array = X_test.to_numpy()
#y_train_array = y_train.to_numpy()
#y_test_array = y_test.to_numpy()
#display(y_train_array)

In [0]:
from sklearn.decomposition import PCA
#create the instance of the model 
pca = PCA(n_components = 2)
#display(spotify_target)
#display(spotify_feature_scaler)
principalComponents = pca.fit_transform(spotify_feature_scaler)
#display(spotify_feature_scaler)
#modified feature set to redeuce dimensions to 2
tempDF = pd.DataFrame(data = principalComponents, columns = ['PCA Feature 1','PCA Feature 2'])
#display(tempDF)
#create a new DF with PCA transformation
PCADF = pd.concat([tempDF.reset_index(), spotify_target.reset_index()], axis = 1)
PCADF = PCADF.drop(['index'], axis=1)
#display(PCADF)

#spotifiy_data
PCAVisual = pd.DataFrame()
genreList = genre_column_list
for genre in genreList:
    #print("Parsing through genre: " + str(genre))
   # PCAVisual=PCAVisual.reset_index()
    tempDF = PCADF.loc[PCADF[genre] == 1] #go through each genre and grab the ones with value 1
    tempDF2 = tempDF[['PCA Feature 1', 'PCA Feature 2', genre]] #will need to change for 3D PCA
    tempDF2.rename(columns={genre: 'Genre'}, inplace=True)
    tempDF2['Genre'].replace({1: genre}, inplace=True)
    PCAVisual= PCAVisual.append(tempDF2)
    #display(PCAVisual)
PCAVisual = PCAVisual.reset_index()
PCAVisual = PCAVisual.drop(['index'], axis =1)
display(PCAVisual)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,PCA Feature 1,PCA Feature 2,Genre
0,-1.319527,1.082352,genre_Movie
1,-1.611358,0.442551,genre_Movie
2,1.513124,-0.297530,genre_Movie
3,1.454878,-0.039260,genre_Movie
4,2.573444,0.132662,genre_Movie
...,...,...,...
232720,-1.633185,-0.019394,genre_World
232721,4.570154,-0.738298,genre_World
232722,-1.234247,-0.265257,genre_World
232723,3.202676,0.303683,genre_World


In [0]:
MLKNN_df = spotify_feature_scaler.copy()
from skmultilearn.adapt import MLkNN
classifier = MLkNN(k=1)
y = spotify_data['genre']
#display()
X = spotify_data[['popularity', 'speechiness']]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

#display(X)
print(len(y))
print(len(X))
X = np.array(X)
y = np.array(y)
#X.columns = X.columns.str.strip() 
#y.columns = y.columns.str.strip()
#for g in genre_column_list:
 #   y.loc[(y['Genre'] == g)] = genre_column_list.index(g) #putting it in numbers of index
#display(y)
#display(X)
#display(y)

In [0]:
def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

In [0]:
from sklearn.svm import SVC
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.decomposition import PCA
model = svm.SVC(kernel='linear')
print(len(X))
clf = model.fit(X, y)

232725


In [0]:
fig, ax = plt.subplots()
# title for the plots
title = ('Decision surface of linear SVC ')
# Set-up grid for plotting.
#print(X['popularity'])
X0, X1 = X[:0], X[:1]
xx, yy = make_meshgrid(X0, X1)

plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_ylabel('PC2')
ax.set_xlabel('PC1')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title('Decison surface using the PCA transformed/projected features')
ax.legend()
plt.show()

In [0]:
from sklearn.metrics import (accuracy_score, preision_score, recall_score)

In [0]:
print(accuracy_score(predictions,y_test_array))
print(precision_score(predictions, y_test_array))
print(recall_score(predictions, y_test_array))

In [0]:
from sklearn.model_selection import GridSearchCV
import time
start_time = time.time()
listK = [21,31,41]
for n in listK: 
    parameters = {'k': [n], 's': [0.0]}
    score = 'f1_macro'

    clf = GridSearchCV(MLkNN(), parameters, scoring=score, verbose = 5)
    clf.fit(X_train_array, y_train_array)

    print (clf.best_params_, clf.best_score_)

elapsed_time = time.time() - start_time

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] k=21, s=0.0 .....................................................
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] k=21, s=0.0 .....................................................
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] k=21, s=0.0 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ......................... k=21, s=0.0, score=0.336, total=16.5min
[CV] k=21, s=0.0 .....................................................
[CV] ......................... k=21, s=0.0, score=0.336, total=16.5min
[CV] k=21, s=0.0 .....................................................
[CV] ......................... k=21, s=0.0, score=0.336, total=16.5min
[CV] k=21, s=0.0 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 16.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 16.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 16.5min remaining:    0.0s


[CV] ......................... k=21, s=0.0, score=0.338, total=15.2min
[CV] k=21, s=0.0 .....................................................
[CV] ......................... k=21, s=0.0, score=0.338, total=15.2min
[CV] k=21, s=0.0 .....................................................
[CV] ......................... k=21, s=0.0, score=0.338, total=15.2min
[CV] k=21, s=0.0 .....................................................


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 67.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 67.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 67.9min remaining:    0.0s


[CV] ......................... k=31, s=0.0, score=0.325, total=17.9min
[CV] ......................... k=31, s=0.0, score=0.325, total=17.9min
[CV] ......................... k=31, s=0.0, score=0.325, total=17.9min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 85.8min finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 85.8min finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 85.8min finished


{'k': 31, 's': 0.0} 0.3241266286279301
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] k=41, s=0.0 .....................................................
{'k': 31, 's': 0.0} 0.3241266286279301
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] k=41, s=0.0 .....................................................
{'k': 31, 's': 0.0} 0.3241266286279301
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] k=41, s=0.0 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
